# Predicting Judicial Decisions of the European Court of Human Rights

In this notebook, we aim to train a classification model to classify cases as 'violation' or 'non-violation'. 
The cases were originally downloaded from HUDOC and structured based on the articles they fall under.

In [1]:
import numpy as np
import re
import os
import copy

In [2]:
import sklearn.metrics as sm
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [3]:
import spacy
from spacy.lang import en

In [4]:
scoring = {'accuracy': make_scorer(sm.accuracy_score),
           'precision': make_scorer(sm.precision_score),
           'recall': make_scorer(sm.recall_score),
           'f1': make_scorer(sm.f1_score)}

To read our dataset, we use os.walk to walk through a sub-tree of directories and files and load all of our training data and labels. We avoid the folder 'both' as the files inside are labelled both as violation and non-violation.
Our data set will be loaded into dictionaries, the keys corresponding to articles and the values will be a list of cases (X - our training set) or labels (Y).

In [5]:
def read_dataset(PATH):
    X_dataset = {}
    Y_dataset = {}
    for path, dirs, files in os.walk(PATH):
        for filename in files:
            fullpath = os.path.join(path, filename)
            if "both" not in fullpath:
                with open(fullpath, 'r', encoding="utf8") as file:
                    X_dataset, Y_dataset = add_file_to_dataset(fullpath, X_dataset, Y_dataset, file.read())

    return X_dataset, Y_dataset       

In [6]:
def add_file_to_dataset(fullpath, x_dataset, y_dataset, file):
    article = extract_article(fullpath)
    file = preprocess(file)
    if article not in x_dataset.keys() :
        x_dataset[article] = []
        y_dataset[article] = []
    x_dataset[article] = x_dataset[article] + [file]
    label = 0 if "non-violation" in fullpath else 1
    y_dataset[article] = y_dataset[article] + [label]
    return x_dataset, y_dataset  

We use regex to extract the number of the Article from the fullpath and insert the file into the list under that specific Article.

In [7]:
def extract_article(path): 
    pattern = r"(Article\d+)"
    result = re.search(pattern, path)
    article = result.group(1)
    return article

### Preprocessing 

Similar to the research paper this work is based on, we will only use the PROCEDURE and THE FACTS paragraphs of the cases as our training set. Otherwise, the model may be biased.

In [8]:
def preprocess(file): 
    file = extract_paragraphs(file)
    return file

In [9]:
def extract_paragraphs(file): 
    file = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', file)
    pat = r'(PROCEDURE\s*\n.+?)?((THE CIRCUMSTANCES OF THE CASE\s*\n.+?RELEVANT DOMESTIC LAW.+?)|(\n(AS TO THE FACTS|THE FACTS)\s*\n.+?))(\nIII\.|THE LAW\s*\n|PROCEEDINGS BEFORE THE COMMISSION\s*\n|ALLEGED VIOLATION OF ARTICLE [0-9]+ OF THE CONVENTION \s*\n)'
    result = re.search(pat, file, re.S |  re.IGNORECASE)
    if result is None:
        print(repr(file))
    content = ""
    if result.group(1) is not None:
        content += result.group(1)
    content += result.group(2)
    return content

### Loading the data

In [10]:
base_path = "Datasets\\Human rights dataset"

In [11]:
X_train_docs, Y_train_docs = read_dataset(base_path + "\\train")
#X_extra_test_docs, Y_extra_test = read_dataset(base_path + "\\test_violations")

In [12]:
X_train_docs.keys()

dict_keys(['Article10', 'Article11', 'Article12', 'Article13', 'Article14', 'Article18', 'Article2', 'Article3', 'Article4', 'Article5', 'Article6', 'Article7', 'Article8'])

Also, similarly to Medvedeva, M., Vols, M. & Wieling, M. Artif Intell Law (2019), we want to remove the articles which contain too few cases. We include Article 11 "as an estimate of how well the model performs when only very few cases are available".

In [13]:
def select_articles(train_set):
    selected_training_set = copy.deepcopy(train_set)
    
    for key in train_set.keys():
        if len(train_set[key]) <= 50:
            selected_training_set.pop(key)
            continue
    return selected_training_set

In [14]:
X_train_docs = select_articles(X_train_docs)

In [15]:
X_train_docs.keys()

dict_keys(['Article10', 'Article11', 'Article13', 'Article14', 'Article2', 'Article3', 'Article5', 'Article6', 'Article8'])

### Combining all the articles according to class

In [16]:
X_train = X_train_docs["Article2"] + X_train_docs["Article3"] + X_train_docs["Article5"] + X_train_docs["Article6"] + X_train_docs["Article8"] + X_train_docs["Article10"] + X_train_docs["Article11"] + X_train_docs["Article13"] + X_train_docs["Article14"]

In [17]:
print(str(len(X_train_docs["Article2"])) + "+" + str(len(X_train_docs["Article3"])) + "+" + str(len(X_train_docs["Article5"])) + "+" + str(len(X_train_docs["Article6"])) + "+" + str(len(X_train_docs["Article8"])) + "+" + str(len(X_train_docs["Article10"])) + "+" + str(len(X_train_docs["Article11"])) + "+" + str(len(X_train_docs["Article13"])) + "+" + str(len(X_train_docs["Article14"])) + "=" + str(len(X_train)))

114+568+300+916+457+212+64+212+288=3131


In [18]:
Y_train = Y_train_docs["Article2"] + Y_train_docs["Article3"] + Y_train_docs["Article5"] + Y_train_docs["Article6"] + Y_train_docs["Article8"] + Y_train_docs["Article10"] + Y_train_docs["Article11"] + Y_train_docs["Article13"] + Y_train_docs["Article14"]

In [19]:
len(Y_train)

3131

### Tokenization with CountVectorizer

In [20]:
# tokenize the doc and lemmatize its tokens
lemmatizer = spacy.lang.en.English()
def my_tokenizer(doc):
    tokens = lemmatizer(doc)
    return([token.lemma_ for token in tokens])

In [21]:
vect = CountVectorizer(ngram_range=(1,2), lowercase=True, tokenizer=my_tokenizer, max_features=700000, binary=True, min_df=3)
term_doc_matrix = vect.fit_transform(X_train).toarray()    

In [22]:
print("The number of features: " + str(len(vect.get_feature_names())))

The number of features: 306631


In [23]:
term_doc_matrix.shape

(3131, 306631)

### Feature Selection

In [24]:
X_train_new = SelectKBest(chi2, k=100000).fit_transform(term_doc_matrix, Y_train)

In [25]:
X_train_new.shape

(3131, 100000)

### Classification with Logistic Regression

In [26]:
classifier_instance = LogisticRegression(solver = 'lbfgs', C=0.5, max_iter=800)
scores = cross_validate(classifier_instance, term_doc_matrix, Y_train, cv=10, scoring=scoring)

In [27]:
print("Accuracy: %0.3f" % (scores["test_accuracy"].mean()))
print("Precision: %0.3f" % (scores["test_precision"].mean()))
print("Recall: %0.3f" % (scores["test_recall"].mean()))
print("F1: %0.3f" % (scores["test_f1"].mean()))

Accuracy: 0.706
Precision: 0.712
Recall: 0.694
F1: 0.696


In [28]:
scores["fit_time"].mean(), scores["score_time"].mean()

(407.357695555687, 3.246254014968872)

0.706 with LogisticRegression(solver = 'lbfgs', C=0.5, max_iter=800) and CountVectorizer(ngram_range=(1,2), lowercase=True, tokenizer=my_tokenizer, max_features=700000, binary=True, min_df=3)

### Tokenization with CountVectorizer and Classification with Linear SVC

In [20]:
vect = CountVectorizer(ngram_range=(1,3), lowercase=True, max_features=700000, binary=True, min_df=5)
term_doc_matrix = vect.fit_transform(X_train).toarray()    

In [21]:
print("The number of features: " + str(len(vect.get_feature_names())))

The number of features: 451342


In [22]:
X_train_new = SelectKBest(chi2, k=400000).fit_transform(term_doc_matrix, Y_train)

In [23]:
X_train_new.shape

(3131, 400000)

In [24]:
classifier_instance = svm.LinearSVC(C=0.5, max_iter=1500)  
scores = cross_validate(classifier_instance, term_doc_matrix, Y_train, cv=10, scoring=scoring)

D:\ProgramFiles\Anaconda\envs\nlp\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\ProgramFiles\Anaconda\envs\nlp\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\ProgramFiles\Anaconda\envs\nlp\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\ProgramFiles\Anaconda\envs\nlp\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\ProgramFiles\Anaconda\envs\nlp\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number o

In [25]:
print("Accuracy: %0.3f" % (scores["test_accuracy"].mean()))
print("Precision: %0.3f" % (scores["test_precision"].mean()))
print("Recall: %0.3f" % (scores["test_recall"].mean()))
print("F1: %0.3f" % (scores["test_f1"].mean()))

Accuracy: 0.721
Precision: 0.727
Recall: 0.707
F1: 0.713


In [26]:
scores["fit_time"].mean(), scores["score_time"].mean()

(155.54953446388245, 3.9933305978775024)

0.721 with LinearSVC(C=0.5, max_iter=1500) and CountVectorizer(ngram_range=(1,3), lowercase=True, max_features=400000, binary=True, min_df=5)

### Tokenization with TfIdfVectorizer and Classification with Logistic Regression

In [21]:
vect = TfidfVectorizer(ngram_range=(2,4), lowercase=True, tokenizer=my_tokenizer, max_features=600000, min_df=3)
term_doc_matrix = vect.fit_transform(X_train).toarray()

In [22]:
print("The number of features: " + str(len(vect.get_feature_names())))

The number of features: 600000


In [23]:
X_train_new = SelectKBest(chi2, k=400000).fit_transform(term_doc_matrix, Y_train)

In [24]:
X_train_new.shape

(3131, 400000)

In [25]:
classifier_instance = LogisticRegression()        
scores = cross_validate(classifier_instance, term_doc_matrix, Y_train, cv=10, scoring=scoring)

D:\ProgramFiles\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\ProgramFiles\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\ProgramFiles\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\ProgramFiles\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\ProgramFiles\Anaconda\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to '

In [26]:
print("Accuracy: %0.3f" % (scores["test_accuracy"].mean()))
print("Precision: %0.3f" % (scores["test_precision"].mean()))
print("Recall: %0.3f" % (scores["test_recall"].mean()))
print("F1: %0.3f" % (scores["test_f1"].mean()))

Accuracy: 0.710
Precision: 0.731
Recall: 0.690
F1: 0.697


In [27]:
scores["fit_time"].mean(), scores["score_time"].mean()

(1094.1596326828003, 4.171499824523925)

0.712 accuracy for LogisticRegression(solver = 'saga') with vect = TfidfVectorizer(ngram_range=(2,4), lowercase=True, tokenizer=my_tokenizer, max_features=400000, min_df=3)

### Tokenization with TfIdfVectorizer and Classification with Linear SVC

In [1]:
vect = TfidfVectorizer(ngram_range=(2,4), lowercase=True, max_features=600000, min_df=3)
term_doc_matrix = vect.fit_transform(X_train).toarray()

NameError: name 'TfidfVectorizer' is not defined

In [21]:
print("The number of features: " + str(len(vect.get_feature_names())))

The number of features: 600000


In [22]:
X_train_new = SelectKBest(chi2, k=300000).fit_transform(term_doc_matrix, Y_train)

In [23]:
X_train_new.shape

(3131, 300000)

In [24]:
classifier_instance = svm.LinearSVC(C=0.1, max_iter=1500)        
scores = cross_validate(classifier_instance, X_train_new, Y_train, cv=10, scoring=scoring)

In [25]:
print("Accuracy: %0.3f" % (scores["test_accuracy"].mean()))
print("Precision: %0.3f" % (scores["test_precision"].mean()))
print("Recall: %0.3f" % (scores["test_recall"].mean()))
print("F1: %0.3f" % (scores["test_f1"].mean()))

Accuracy: 0.744
Precision: 0.766
Recall: 0.722
F1: 0.732


In [26]:
scores["fit_time"].mean(), scores["score_time"].mean()

(155.90342404842377, 0.9028446197509765)

0.744 accuracy with LinearSVC(C=0.1, max_iter=1500) and TfidfVectorizer(ngram_range=(2,4), lowercase=True, max_features=700000, min_df=3) with SelectKBest(chi2, k=300000)